In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    balanced_accuracy_score,
    precision_recall_fscore_support,
    confusion_matrix,
    roc_auc_score,
)
import SimpleITK as sitk
import pandas as pd
import numpy as np
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2

In [ ]:
device = torch.device(f"cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
import torch.nn as nn
import timm


model_names = ['inception_v3', 'mobilenetv2_100', 'efficientnetv2_m', 'vgg16']

for model_name in model_names:
    try:
        model = timm.create_model(model_name, pretrained=False)
        default_cfg = model.default_cfg
        
        print(f"--- 모델: {model_name} ---")
        print(f"  예상 입력 크기 (C, H, W): {default_cfg.get('input_size')}")
        print(f"  정규화 평균 (mean): {default_cfg.get('mean')}")
        print(f"  정규화 표준 편차 (std): {default_cfg.get('std')}")
        print("-" * 30)
    except Exception as e:
        print(f"--- 모델: {model_name} ---")
        print(f"  정보를 가져오는 데 실패했습니다: {e}")
        print("-" * 30)

--- 모델: inception_v3 ---
  예상 입력 크기 (C, H, W): (3, 299, 299)
  정규화 평균 (mean): (0.5, 0.5, 0.5)
  정규화 표준 편차 (std): (0.5, 0.5, 0.5)
------------------------------
--- 모델: mobilenetv2_100 ---
  예상 입력 크기 (C, H, W): (3, 224, 224)
  정규화 평균 (mean): (0.485, 0.456, 0.406)
  정규화 표준 편차 (std): (0.229, 0.224, 0.225)
------------------------------
--- 모델: efficientnetv2_m ---
  예상 입력 크기 (C, H, W): (3, 320, 320)
  정규화 평균 (mean): (0.485, 0.456, 0.406)
  정규화 표준 편차 (std): (0.229, 0.224, 0.225)
------------------------------
--- 모델: vgg16 ---
  예상 입력 크기 (C, H, W): (3, 224, 224)
  정규화 평균 (mean): (0.485, 0.456, 0.406)
  정규화 표준 편차 (std): (0.229, 0.224, 0.225)
------------------------------


In [6]:
from mydata import create_dataloader_patch

In [ ]:
model = timm.create_model('resnet50', pretrained=True, num_classes=2).to(device)

batch_size = 8
train_loader, val_loader = create_dataloader_patch(batch_size=batch_size)

num_epochs = 75

criterion = nn.CrossEntropyLoss(label_smoothing=0.05)
optimizer = optim.SGD(model.parameters(), lr=5e-3, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5 * len(train_loader), T_mult=2)

train_losses, val_losses = [], []
train_accs,   val_accs   = [], []      # plain accuracy
bal_accs, aucs = [], []                # extra metrics

best_bal_acc = 90.0                     # (optional) best-model 저장용

for epoch in range(1, num_epochs + 1):
    # ---------------- TRAIN ----------------
    model.train()
    running_loss, running_correct, running_total = 0.0, 0, 0

    for x, y in train_loader:
        x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
        optimizer.zero_grad()
        
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

        # LR sched : **batch level**
        if scheduler is not None:
            scheduler.step()

        running_loss   += loss.item()
        running_correct += (logits.argmax(1) == y).sum().item()
        running_total  += y.size(0)

    avg_train_loss = running_loss / len(train_loader)
    train_acc      = 100. * running_correct / running_total
    train_losses.append(avg_train_loss)
    train_accs  .append(train_acc)

    # ---------------- VALIDATION ----------------
    model.eval()
    val_loss, val_logits, val_labels = 0.0, [], []

    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
            logits = model(x)
            loss = criterion(logits, y)

            val_loss   += loss.item()
            val_logits.append(logits.cpu())
            val_labels.append(y.cpu())

    val_logits = torch.cat(val_logits)
    val_labels = torch.cat(val_labels)
    probs = F.softmax(val_logits, dim=1).numpy()
    preds      = val_logits.argmax(1).numpy()
    labels     = val_labels.numpy()

    # --- metrics ---
    avg_val_loss = val_loss / len(val_loader)
    val_acc_plain = accuracy_score(labels, preds) * 100
    bal_acc   = balanced_accuracy_score(labels, preds) * 100
    
    prec, rec, f1, _ = precision_recall_fscore_support(
        labels, preds, zero_division=0
    )
    cm = confusion_matrix(labels, preds)

    # 기록
    val_losses.append(avg_val_loss)
    val_accs  .append(val_acc_plain)
    bal_accs  .append(bal_acc)
    # aucs      .append(auc)

    # 로그 출력 -------------------------------------------------------
    print(
        f"[Epoch {epoch:03d}/{num_epochs:03d}] "
        f"TrainLoss {avg_train_loss:.4f} | "
        f"ValLoss {avg_val_loss:.4f} | "
        f"Acc {val_acc_plain:.2f}% | "
        f"BalAcc {bal_acc:.2f}% | ",
        # f"AUC {auc:.3f}",
        flush=True
    )
    
    if bal_acc > best_bal_acc:
        best_bal_acc = bal_acc
        print(f'best bal acc : {bal_acc:.2f}')


7747 1937
[Epoch 001/075] TrainLoss 0.6080 | ValLoss 0.5461 | Acc 76.05% | BalAcc 75.27% | 
[Epoch 002/075] TrainLoss 0.5464 | ValLoss 0.5299 | Acc 76.15% | BalAcc 75.20% | 
[Epoch 003/075] TrainLoss 0.5196 | ValLoss 0.5114 | Acc 77.39% | BalAcc 76.79% | 
[Epoch 004/075] TrainLoss 0.5052 | ValLoss 0.4532 | Acc 81.78% | BalAcc 81.49% | 
[Epoch 005/075] TrainLoss 0.4996 | ValLoss 0.4569 | Acc 82.60% | BalAcc 82.59% | 
[Epoch 006/075] TrainLoss 0.5129 | ValLoss 0.4928 | Acc 78.32% | BalAcc 77.56% | 
[Epoch 007/075] TrainLoss 0.5075 | ValLoss 0.4573 | Acc 82.19% | BalAcc 82.18% | 
[Epoch 008/075] TrainLoss 0.4906 | ValLoss 0.4470 | Acc 82.71% | BalAcc 82.59% | 
[Epoch 009/075] TrainLoss 0.4812 | ValLoss 0.4457 | Acc 82.50% | BalAcc 82.42% | 
[Epoch 010/075] TrainLoss 0.4772 | ValLoss 0.4331 | Acc 83.38% | BalAcc 83.14% | 
[Epoch 011/075] TrainLoss 0.4695 | ValLoss 0.4249 | Acc 83.01% | BalAcc 82.90% | 
[Epoch 012/075] TrainLoss 0.4655 | ValLoss 0.4217 | Acc 83.48% | BalAcc 83.34% | 
[Epoch

In [ ]:
model = timm.create_model('vgg16', pretrained=True, num_classes=2).to(device)

batch_size = 8
train_loader, val_loader = create_dataloader_patch(batch_size=batch_size)

num_epochs = 75

criterion = nn.CrossEntropyLoss(label_smoothing=0.05)
optimizer = optim.SGD(model.parameters(), lr=1e-4, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5 * len(train_loader), T_mult=2)

train_losses, val_losses = [], []
train_accs,   val_accs   = [], []      # plain accuracy
bal_accs, aucs = [], []                # extra metrics

best_bal_acc = 90.0                     # (optional) best-model 저장용

for epoch in range(1, num_epochs + 1):
    # ---------------- TRAIN ----------------
    model.train()
    running_loss, running_correct, running_total = 0.0, 0, 0

    for x, y in train_loader:
        x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
        optimizer.zero_grad()
        
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

        # LR sched : **batch level**
        if scheduler is not None:
            scheduler.step()

        running_loss   += loss.item()
        running_correct += (logits.argmax(1) == y).sum().item()
        running_total  += y.size(0)

    avg_train_loss = running_loss / len(train_loader)
    train_acc      = 100. * running_correct / running_total
    train_losses.append(avg_train_loss)
    train_accs  .append(train_acc)

    # ---------------- VALIDATION ----------------
    model.eval()
    val_loss, val_logits, val_labels = 0.0, [], []

    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
            logits = model(x)
            loss = criterion(logits, y)

            val_loss   += loss.item()
            val_logits.append(logits.cpu())
            val_labels.append(y.cpu())

    val_logits = torch.cat(val_logits)
    val_labels = torch.cat(val_labels)
    probs = F.softmax(val_logits, dim=1).numpy()
    preds      = val_logits.argmax(1).numpy()
    labels     = val_labels.numpy()

    # --- metrics ---
    avg_val_loss = val_loss / len(val_loader)
    val_acc_plain = accuracy_score(labels, preds) * 100
    bal_acc   = balanced_accuracy_score(labels, preds) * 100
    
    prec, rec, f1, _ = precision_recall_fscore_support(
        labels, preds, zero_division=0
    )
    cm = confusion_matrix(labels, preds)

    # 기록
    val_losses.append(avg_val_loss)
    val_accs  .append(val_acc_plain)
    bal_accs  .append(bal_acc)
    # aucs      .append(auc)

    # 로그 출력 -------------------------------------------------------
    print(
        f"[Epoch {epoch:03d}/{num_epochs:03d}] "
        f"TrainLoss {avg_train_loss:.4f} | "
        f"ValLoss {avg_val_loss:.4f} | "
        f"Acc {val_acc_plain:.2f}% | "
        f"BalAcc {bal_acc:.2f}% | ",
        # f"AUC {auc:.3f}",
        flush=True
    )

    if bal_acc > best_bal_acc:
        best_bal_acc = bal_acc
        print(f'best bal acc : {bal_acc:.2f}')


7747 1937
[Epoch 001/075] TrainLoss 0.5818 | ValLoss 0.5086 | Acc 79.45% | BalAcc 79.76% | 
[Epoch 002/075] TrainLoss 0.5131 | ValLoss 0.4454 | Acc 83.01% | BalAcc 82.93% | 
[Epoch 003/075] TrainLoss 0.4906 | ValLoss 0.4328 | Acc 83.43% | BalAcc 83.13% | 
[Epoch 004/075] TrainLoss 0.4729 | ValLoss 0.4286 | Acc 84.00% | BalAcc 83.98% | 
[Epoch 005/075] TrainLoss 0.4682 | ValLoss 0.4230 | Acc 84.41% | BalAcc 84.29% | 
[Epoch 006/075] TrainLoss 0.4799 | ValLoss 0.4381 | Acc 82.81% | BalAcc 82.30% | 
[Epoch 007/075] TrainLoss 0.4646 | ValLoss 0.4415 | Acc 83.48% | BalAcc 83.75% | 
[Epoch 008/075] TrainLoss 0.4492 | ValLoss 0.3975 | Acc 86.32% | BalAcc 86.16% | 
[Epoch 009/075] TrainLoss 0.4416 | ValLoss 0.3966 | Acc 86.32% | BalAcc 86.07% | 
[Epoch 010/075] TrainLoss 0.4333 | ValLoss 0.3875 | Acc 86.63% | BalAcc 86.66% | 
[Epoch 011/075] TrainLoss 0.4240 | ValLoss 0.3744 | Acc 87.25% | BalAcc 87.13% | 
[Epoch 012/075] TrainLoss 0.4152 | ValLoss 0.3729 | Acc 87.56% | BalAcc 87.35% | 
[Epoch

In [ ]:
model = timm.create_model('efficientnetv2_m', pretrained=False, num_classes=2).to(device)

batch_size = 8
train_loader, val_loader = create_dataloader_patch(batch_size=batch_size, out_size=320)

num_epochs = 75

criterion = nn.CrossEntropyLoss(label_smoothing=0.05)
optimizer = optim.SGD(model.parameters(), lr=4e-3, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5 * len(train_loader), T_mult=2)

train_losses, val_losses = [], []
train_accs,   val_accs   = [], []      # plain accuracy
bal_accs, aucs = [], []                # extra metrics

best_bal_acc = 90.0                     # (optional) best-model 저장용

for epoch in range(1, num_epochs + 1):
    # ---------------- TRAIN ----------------
    model.train()
    running_loss, running_correct, running_total = 0.0, 0, 0

    for x, y in train_loader:
        x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
        optimizer.zero_grad()
        
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

        # LR sched : **batch level**
        if scheduler is not None:
            scheduler.step()

        running_loss   += loss.item()
        running_correct += (logits.argmax(1) == y).sum().item()
        running_total  += y.size(0)

    avg_train_loss = running_loss / len(train_loader)
    train_acc      = 100. * running_correct / running_total
    train_losses.append(avg_train_loss)
    train_accs  .append(train_acc)

    # ---------------- VALIDATION ----------------
    model.eval()
    val_loss, val_logits, val_labels = 0.0, [], []

    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
            logits = model(x)
            loss = criterion(logits, y)

            val_loss   += loss.item()
            val_logits.append(logits.cpu())
            val_labels.append(y.cpu())

    val_logits = torch.cat(val_logits)
    val_labels = torch.cat(val_labels)
    probs = F.softmax(val_logits, dim=1).numpy()
    preds      = val_logits.argmax(1).numpy()
    labels     = val_labels.numpy()

    # --- metrics ---
    avg_val_loss = val_loss / len(val_loader)
    val_acc_plain = accuracy_score(labels, preds) * 100
    bal_acc   = balanced_accuracy_score(labels, preds) * 100
    
    prec, rec, f1, _ = precision_recall_fscore_support(
        labels, preds, zero_division=0
    )
    cm = confusion_matrix(labels, preds)

    # 기록
    val_losses.append(avg_val_loss)
    val_accs  .append(val_acc_plain)
    bal_accs  .append(bal_acc)
    # aucs      .append(auc)

    # 로그 출력 -------------------------------------------------------
    print(
        f"[Epoch {epoch:03d}/{num_epochs:03d}] "
        f"TrainLoss {avg_train_loss:.4f} | "
        f"ValLoss {avg_val_loss:.4f} | "
        f"Acc {val_acc_plain:.2f}% | "
        f"BalAcc {bal_acc:.2f}% | ",
        # f"AUC {auc:.3f}",
        flush=True
    )
    
    if bal_acc > best_bal_acc:
        best_bal_acc = bal_acc
        print(f'best bal acc : {bal_acc:.2f}')


7747 1937
[Epoch 001/075] TrainLoss 0.8358 | ValLoss 0.5284 | Acc 78.16% | BalAcc 77.59% | 
[Epoch 002/075] TrainLoss 0.5555 | ValLoss 0.4643 | Acc 82.03% | BalAcc 81.73% | 
[Epoch 003/075] TrainLoss 0.5206 | ValLoss 0.4494 | Acc 82.03% | BalAcc 81.92% | 
[Epoch 004/075] TrainLoss 0.4923 | ValLoss 0.4266 | Acc 82.76% | BalAcc 82.45% | 
[Epoch 005/075] TrainLoss 0.4721 | ValLoss 0.4226 | Acc 82.55% | BalAcc 82.22% | 
[Epoch 006/075] TrainLoss 0.5441 | ValLoss 0.4424 | Acc 82.45% | BalAcc 82.27% | 
[Epoch 007/075] TrainLoss 0.5144 | ValLoss 0.4393 | Acc 82.34% | BalAcc 82.40% | 
[Epoch 008/075] TrainLoss 0.4935 | ValLoss 0.4330 | Acc 82.76% | BalAcc 82.47% | 
[Epoch 009/075] TrainLoss 0.4795 | ValLoss 0.4039 | Acc 83.22% | BalAcc 82.98% | 
[Epoch 010/075] TrainLoss 0.4668 | ValLoss 0.4037 | Acc 84.36% | BalAcc 84.27% | 
[Epoch 011/075] TrainLoss 0.4524 | ValLoss 0.3797 | Acc 85.13% | BalAcc 84.78% | 
[Epoch 012/075] TrainLoss 0.4279 | ValLoss 0.3601 | Acc 87.61% | BalAcc 87.44% | 
[Epoch

In [ ]:
model = timm.create_model('mobilenetv2_100', pretrained=True, num_classes=2).to(device)

batch_size = 8
train_loader, val_loader = create_dataloader_patch(batch_size=batch_size)

num_epochs = 75

criterion = nn.CrossEntropyLoss(label_smoothing=0.05)
optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5 * len(train_loader), T_mult=2)

train_losses, val_losses = [], []
train_accs,   val_accs   = [], []      # plain accuracy
bal_accs, aucs = [], []                # extra metrics

best_bal_acc = 90.0                     # (optional) best-model 저장용

for epoch in range(1, num_epochs + 1):
    # ---------------- TRAIN ----------------
    model.train()
    running_loss, running_correct, running_total = 0.0, 0, 0

    for x, y in train_loader:
        x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
        optimizer.zero_grad()
        
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

        # LR sched : **batch level**
        if scheduler is not None:
            scheduler.step()

        running_loss   += loss.item()
        running_correct += (logits.argmax(1) == y).sum().item()
        running_total  += y.size(0)

    avg_train_loss = running_loss / len(train_loader)
    train_acc      = 100. * running_correct / running_total
    train_losses.append(avg_train_loss)
    train_accs  .append(train_acc)

    # ---------------- VALIDATION ----------------
    model.eval()
    val_loss, val_logits, val_labels = 0.0, [], []

    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
            logits = model(x)
            loss = criterion(logits, y)

            val_loss   += loss.item()
            val_logits.append(logits.cpu())
            val_labels.append(y.cpu())

    val_logits = torch.cat(val_logits)
    val_labels = torch.cat(val_labels)
    probs = F.softmax(val_logits, dim=1).numpy()
    preds      = val_logits.argmax(1).numpy()
    labels     = val_labels.numpy()

    # --- metrics ---
    avg_val_loss = val_loss / len(val_loader)
    val_acc_plain = accuracy_score(labels, preds) * 100
    bal_acc   = balanced_accuracy_score(labels, preds) * 100
    
    prec, rec, f1, _ = precision_recall_fscore_support(
        labels, preds, zero_division=0
    )
    cm = confusion_matrix(labels, preds)

    # 기록
    val_losses.append(avg_val_loss)
    val_accs  .append(val_acc_plain)
    bal_accs  .append(bal_acc)
    # aucs      .append(auc)

    # 로그 출력 -------------------------------------------------------
    print(
        f"[Epoch {epoch:03d}/{num_epochs:03d}] "
        f"TrainLoss {avg_train_loss:.4f} | "
        f"ValLoss {avg_val_loss:.4f} | "
        f"Acc {val_acc_plain:.2f}% | "
        f"BalAcc {bal_acc:.2f}% | ",
        # f"AUC {auc:.3f}",
        flush=True
    )
    
    if bal_acc > best_bal_acc:
        best_bal_acc = bal_acc
        print(f'best bal acc : {bal_acc:.2f}')


model.safetensors:   0%|          | 0.00/14.2M [00:00<?, ?B/s]

7747 1937
[Epoch 001/075] TrainLoss 0.7100 | ValLoss 0.4731 | Acc 81.21% | BalAcc 81.13% | 
[Epoch 002/075] TrainLoss 0.5343 | ValLoss 0.4783 | Acc 81.88% | BalAcc 81.89% | 
[Epoch 003/075] TrainLoss 0.5130 | ValLoss 0.4536 | Acc 82.03% | BalAcc 81.91% | 
[Epoch 004/075] TrainLoss 0.4954 | ValLoss 0.4504 | Acc 82.40% | BalAcc 82.23% | 
[Epoch 005/075] TrainLoss 0.4837 | ValLoss 0.4515 | Acc 82.55% | BalAcc 82.45% | 
[Epoch 006/075] TrainLoss 0.5080 | ValLoss 0.4571 | Acc 82.29% | BalAcc 82.29% | 
[Epoch 007/075] TrainLoss 0.4946 | ValLoss 0.4378 | Acc 82.86% | BalAcc 82.59% | 
[Epoch 008/075] TrainLoss 0.4884 | ValLoss 0.4421 | Acc 82.91% | BalAcc 82.96% | 
[Epoch 009/075] TrainLoss 0.4775 | ValLoss 0.4354 | Acc 83.58% | BalAcc 83.46% | 
[Epoch 010/075] TrainLoss 0.4724 | ValLoss 0.4240 | Acc 84.31% | BalAcc 84.06% | 
[Epoch 011/075] TrainLoss 0.4652 | ValLoss 0.4227 | Acc 83.43% | BalAcc 83.10% | 
[Epoch 012/075] TrainLoss 0.4627 | ValLoss 0.4144 | Acc 84.20% | BalAcc 84.04% | 
[Epoch

In [ ]:
model = timm.create_model('inception_v3', pretrained=True, num_classes=2).to(device)

batch_size = 8
train_loader, val_loader = create_dataloader_patch(batch_size=batch_size, out_size=299)

num_epochs = 75

criterion = nn.CrossEntropyLoss(label_smoothing=0.05)
optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5 * len(train_loader), T_mult=2)

train_losses, val_losses = [], []
train_accs,   val_accs   = [], []      # plain accuracy
bal_accs, aucs = [], []                # extra metrics

best_bal_acc = 90.0                     # (optional) best-model 저장용

for epoch in range(1, num_epochs + 1):
    # ---------------- TRAIN ----------------
    model.train()
    running_loss, running_correct, running_total = 0.0, 0, 0

    for x, y in train_loader:
        x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
        optimizer.zero_grad()
        
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

        # LR sched : **batch level**
        if scheduler is not None:
            scheduler.step()

        running_loss   += loss.item()
        running_correct += (logits.argmax(1) == y).sum().item()
        running_total  += y.size(0)

    avg_train_loss = running_loss / len(train_loader)
    train_acc      = 100. * running_correct / running_total
    train_losses.append(avg_train_loss)
    train_accs  .append(train_acc)

    # ---------------- VALIDATION ----------------
    model.eval()
    val_loss, val_logits, val_labels = 0.0, [], []

    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
            logits = model(x)
            loss = criterion(logits, y)

            val_loss   += loss.item()
            val_logits.append(logits.cpu())
            val_labels.append(y.cpu())

    val_logits = torch.cat(val_logits)
    val_labels = torch.cat(val_labels)
    probs = F.softmax(val_logits, dim=1).numpy()
    preds      = val_logits.argmax(1).numpy()
    labels     = val_labels.numpy()

    # --- metrics ---
    avg_val_loss = val_loss / len(val_loader)
    val_acc_plain = accuracy_score(labels, preds) * 100
    bal_acc   = balanced_accuracy_score(labels, preds) * 100
    
    prec, rec, f1, _ = precision_recall_fscore_support(
        labels, preds, zero_division=0
    )
    cm = confusion_matrix(labels, preds)

    # 기록
    val_losses.append(avg_val_loss)
    val_accs  .append(val_acc_plain)
    bal_accs  .append(bal_acc)
    # aucs      .append(auc)

    # 로그 출력 -------------------------------------------------------
    print(
        f"[Epoch {epoch:03d}/{num_epochs:03d}] "
        f"TrainLoss {avg_train_loss:.4f} | "
        f"ValLoss {avg_val_loss:.4f} | "
        f"Acc {val_acc_plain:.2f}% | "
        f"BalAcc {bal_acc:.2f}% | ",
        # f"AUC {auc:.3f}",
        flush=True
    )

    if bal_acc > best_bal_acc:
        best_bal_acc = bal_acc
        print(f'best bal acc : {bal_acc:.2f}')


7747 1937
[Epoch 001/075] TrainLoss 0.6104 | ValLoss 0.4998 | Acc 79.97% | BalAcc 80.33% | 
[Epoch 002/075] TrainLoss 0.5106 | ValLoss 0.4666 | Acc 80.80% | BalAcc 80.06% | 
[Epoch 003/075] TrainLoss 0.4779 | ValLoss 0.4119 | Acc 84.87% | BalAcc 84.78% | 
[Epoch 004/075] TrainLoss 0.4500 | ValLoss 0.3962 | Acc 85.60% | BalAcc 85.53% | 
[Epoch 005/075] TrainLoss 0.4363 | ValLoss 0.3914 | Acc 86.58% | BalAcc 86.49% | 
[Epoch 006/075] TrainLoss 0.4762 | ValLoss 0.4200 | Acc 84.15% | BalAcc 83.75% | 
[Epoch 007/075] TrainLoss 0.4555 | ValLoss 0.4005 | Acc 85.18% | BalAcc 85.08% | 
[Epoch 008/075] TrainLoss 0.4474 | ValLoss 0.3936 | Acc 85.54% | BalAcc 85.32% | 
[Epoch 009/075] TrainLoss 0.4263 | ValLoss 0.4151 | Acc 83.84% | BalAcc 83.22% | 
[Epoch 010/075] TrainLoss 0.4161 | ValLoss 0.3593 | Acc 87.56% | BalAcc 87.29% | 
[Epoch 011/075] TrainLoss 0.4034 | ValLoss 0.3429 | Acc 88.59% | BalAcc 88.39% | 
[Epoch 012/075] TrainLoss 0.3894 | ValLoss 0.3348 | Acc 89.26% | BalAcc 89.05% | 
[Epoch